In [5]:
## datadam_sales_per_lease

## 확인사항
## 1. KT데이터 업데이트 여부
## 2. 건축물대장 업데이트 여부
## 3. 부동산원 임대동향조사 업데이트 여부

# module
import psycopg2
import pandas as pd

# 기준월
# date = '202210'
dates = pd.date_range('2022-07',periods=4,freq='M').strftime("%Y%m")
quarter = '202203'

# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

for date in dates:
    # INSERT
    aws_cursor.execute(
    """
    insert into m2.datadam_sales_per_lease (
    with date as (
        select '""" + date + """'::text as base_ym
    )
    select (select base_ym from date) as base_ym
         , A.pnu
         , substring(A.pnu, 1, 8) as emd_cd
         , industry_cd
         , round(A.sales_per_area / B.lease_per_area, 2) as estimate_sales
         , sysdate::date as create_at
         , sysdate::date as update_at
         , 'du.Park' as work_user
    from (
        select T1.pnu
             , T1.industry_cd
             , round(1.0 * T1.sales / 33, 0) as sales_per_area
        from (
            select substring(bd_mgt_sn, 1, 19) as pnu
                 , ftc_cate2_cd as industry_cd
                 , avg(pred_slng_amt) as sales
            from m1.kt_bldg_sales 
            where 1=1
                and base_ym = (select base_ym from date) -- 기준월
            group by 1, 2
        ) T1
        join m2.cremao_land T2
            on T1.pnu = T2.pnu
        join (
            select distinct sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu
            from m1.bld_rgst
            where 1=1
                and base_ym = (select base_ym from date) -- 기준월
        ) T3
            on T1.pnu = T3.pnu
    ) A
    left join (
        -- 시도단위 임대료 데이터 (부동산원 데이터_분기단위)
        select T1.sidocode as sido_cd
             , round(avg(T2.lease) * 1000 * 3.3, 0) as lease_per_area -- 3.3 제곱미터 단위
        from m1.polygon_area T1
        left join (
            select region_nm
                 , avg(rent_fee) as lease
            from m1.lease_price
            where research_date = '""" + quarter + """'
            group by 1
        ) T2
            on T1.cname = T2.region_nm
        group by 1
    ) B
        on substring(A.pnu, 1, 2) = B.sido_cd
    );
    """
    )
    aws_conn.commit()
    print("Done")

Done
Done
Done
Done


###### ver1

In [ ]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# PNU별 매출액
sql = '''

'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df = pd.DataFrame(rows)
df.columns = ['pnu','emd_cd','industry_cd','sales_per_lease']

# 작업일(업데이트일) 및 작업자 추가
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'

In [ ]:
### 2. DB INSERT
import datetime

table = 'm2.datadam_sales_per_lease'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))